# Practice of descriptors

In [8]:
# Imports
import datetime

In [23]:
class TimeUtc:
    def __get__(self, instance, owner_class):
        print(instance, owner_class)
        # return datetime.utcnow().isoformat()
        return datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%d %H:%M:%S")



In [24]:
class Logger:
    current_time = TimeUtc()

In [25]:
l = Logger()
l.current_time

<__main__.Logger object at 0x7fd1f428c050> <class '__main__.Logger'>


'2024-09-16 07:29:03'

In [26]:
Logger.current_time

None <class '__main__.Logger'>


'2024-09-16 07:29:06'

In [27]:
l

## Caveat with class attributes
When we create the non data, data descriptors like this, we found that the variable is not assigned to the instance itself, instead it is assigned at the class level. When we create 2 instances of the same descriptor we will face an issue because the value will be overwritten

In [28]:
class TimeUtc:
    def __get__(self, instance, owner_class):
        print(instance, owner_class)
        # return datetime.utcnow().isoformat()
        # If the instance is none, means the get method was called from the class, then we return the descriptor instance itself.
        if instance is None:
            return self
        return datetime.datetime.now(datetime.UTC).strftime("%Y-%m-%d %H:%M:%S")

In [30]:
class Logger:
    current_time = TimeUtc()

In [31]:
l = Logger()

In [33]:
Logger.current_time
# this returns the instance of the descriptor

None <class '__main__.Logger'>


l.current_time

## Using descriptors as instance properties

## Using descriptors as instance properties

In [14]:
class IntegerValueDescriptor:
    def __init__(self, name):
        self.storage_name = f'_{name}'

    def __set__(self, instance, value):
        setattr(instance, self.storage_name, int(value))

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        return getattr(instance, self.storage_name,None)
        

In [15]:
class Point2D:
    x = IntegerValueDescriptor('x')
    y = IntegerValueDescriptor('y')

In [16]:
p1 = Point2D()
p1.x = 10.1
p1.y = 20.2

In [17]:
p1.__dict__

{'_x': 10, '_y': 20}

In [18]:
p1.x, p1.y

(10, 20)

## Set name method

In [49]:
class ValidString:
    def __set_name__(self, owner_class, property_name):
        self.property_name = property_name

    def __set__(self, instance, value):
        # setattr(instance, self.property_name, value)
        instance.__dict__[self.property_name] = value

    def __get__(self, instance, owner_class):
        if instance is None:
            return self
        # return getattr(instance, self.property_name, None)
        return instance.__dict__.get(self.property_name)

In [50]:
class Person:
    name = ValidString()

In [51]:
p = Person()

In [53]:
Person.name = 'Andres'

In [54]:
print(p.name)

Andres
